In [16]:
import os 
os.chdir(r"C:\Users\prabh\Desktop\MEDBOT-GEN\Data")

# Confirm current location
%pwd  # Should list your PDF files

'C:\\Users\\prabh\\Desktop\\MEDBOT-GEN\\Data'

In [17]:
os.chdir('../')
%pwd

'C:\\Users\\prabh\\Desktop\\MEDBOT-GEN'

In [18]:
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [19]:
def  load_pdf_file(data):
    loader=DirectoryLoader(data,
                           glob="*.pdf",
                           loader_cls=PyPDFLoader)
    documents=loader.load()
    return documents

In [20]:
extracted_data=load_pdf_file(data="Data/")

In [21]:
def text_splitter(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [22]:
text_chunks=text_splitter(extracted_data)
# text_chunks

In [23]:
!pip install -U sentence-transformers huggingface-hub==0.19.4


^C


  Using cached sentence_transformers-5.0.0-py3-none-any.whl.metadata (16 kB)
  Using cached transformers-4.53.3-py3-none-any.whl.metadata (40 kB)
INFO: pip is looking at multiple versions of sentence-transformers to determine which version is compatible with other requirements. This could take a while.
  Using cached sentence_transformers-4.1.0-py3-none-any.whl.metadata (13 kB)
  Using cached sentence_transformers-4.0.2-py3-none-any.whl.metadata (13 kB)
  Using cached sentence_transformers-4.0.1-py3-none-any.whl.metadata (13 kB)
  Using cached sentence_transformers-4.0.0-py3-none-any.whl.metadata (13 kB)
  Using cached sentence_transformers-3.4.1-py3-none-any.whl.metadata (10 kB)
  Using cached sentence_transformers-3.4.0-py3-none-any.whl.metadata (10 kB)
  Using cached sentence_transformers-3.3.1-py3-none-any.whl.metadata (10 kB)
INFO: pip is still looking at multiple versions of sentence-transformers to determine which version is compatible with other requirements. This could take a 

In [24]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")
emb = model.encode("Medical AI is evolving.")
print(emb[:5])


[ 0.01223582 -0.02567114 -0.00795485 -0.01180924  0.00740596]


In [27]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
def download_hugging_face_embeddings():
    embeddings = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [29]:
embed=download_hugging_face_embeddings()

In [ ]:
# query=embed.embed_query("hello world")
# print(len(query))

384


In [30]:
from dotenv import load_dotenv
load_dotenv()

False

In [32]:
import os
PINECONE_KEY=os.environ.get('pinecone_key')


In [45]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

pc=Pinecone(api_key=PINECONE_KEY)

index_name="medicobot"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

{
    "name": "medicobot",
    "metric": "cosine",
    "host": "medicobot-y024lfp.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [62]:
import os

os.environ["PINECONE_API_KEY"] = "pcsk_4Cfm7C_Bxz4uqNrxBsEX351yhNbEU9o1RnJq5Znz5fZYG5MnEHd76y2jmVZ3UbTfwvvyng"
os.environ["PINECONE_ENVIRONMENT"] = "us-east-1"  # Not always needed, but helps

# Optional: define host explicitly if using Serverless
os.environ["PINECONE_HOST"] = "medicobot-y024lfp.svc.aped-4627-b74a.pinecone.io"


In [63]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key="pcsk_4Cfm7C_Bxz4uqNrxBsEX351yhNbEU9o1RnJq5Znz5fZYG5MnEHd76y2jmVZ3UbTfwvvyng")

# Optional: verify index
print(pc.list_indexes())


[{
    "name": "medicobot",
    "metric": "cosine",
    "host": "medicobot-y024lfp.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}]


In [64]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embed,
)


Index host ignored when initializing with index object.


In [65]:
from langchain_pinecone import PineconeVectorStore

docsearch=PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embed
)

Index host ignored when initializing with index object.


In [66]:
docsearch

In [67]:
retriever=docsearch.as_retriever(search_type="similarity",search_kwargs={"k":3})

In [68]:
retrieved_docs=retriever.invoke("what is Acne ?")

In [69]:
retrieved_docs

[Document(id='73873e53-6a0f-4e1c-88de-42148e60fb69', metadata={'creationdate': '2004-12-18T17:52:16-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T18:38:56-06:00', 'page': 624.0, 'page_label': '625', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\medical_book.pdf', 'total_pages': 940.0}, page_content='Helm, Klaus F., and James G. Marks, Jr. Atlas of Differential\nDiagnosis in Dermatology. New York: Churchill Living-\nstone, 1998.\nMacsai, Marian S., et al. “Acne Rosacea.” In Eye and Skin Dis-\nease, ed. Mark J. Mannis, et al. Philadelphia: Lippincott-\nRaven, 1996.\nPERIODICALS\nJansen, Thomas, and Gerd Plewig. “Rosacea: Classification\nand Treatment.”Journal of the Royal Society of Medicine\n90 (Mar. 1997): 144-150.\nThiboutot, Diane M. “Acne Rosacea.”American Family Physi-'),
 Document(id='5fe185ec-bd5d-4ce8-b039-82853a24afff', metadata={'creationdate': '2004-12-18T17:52:16-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T18:38:56-06:00', 'page': 685.0, 'page_label': 